<a href="https://colab.research.google.com/github/LeoDardanello/MultiModalXAI/blob/main/xai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [4]:
from transformers import CLIPProcessor, CLIPModel

from PIL import Image # per debug
import requests # per debug

from torch import nn

In [5]:
# Pretrained CLIP loading...
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
class MyModel(nn.Module):
  def __init__(self, num_linear_layers,task_a_out=1, task_b_out=4, input_dim=1024, hidden_dim=512, drop_value=0.2): # input_dim = 1024 supposing that late fusion among embeddings is adopted
    super().__init__()
    self.head_task_a = nn.Linear(hidden_dim, task_a_out)
    self.head_task_b = nn.Linear(hidden_dim, task_b_out)

    # da implementare il preprocess per il late fusion
    self.layers = []

    # add normalization
    for i in range(num_linear_layers):
      if i == 0:
        self.layers.append(nn.Linear(input_dim, hidden_dim))
      else:
        self.layers.append(nn.Linear(hidden_dim, hidden_dim))

      self.layers.append(nn.Dropout(drop_value))
      self.layers.append(nn.ReLU())


  def forward(self, x):
    for layer in self.layers:
      x = layer(x)

    return self.head_task_a(x), self.head_task_b(x)

In [ ]:
# testing the model...
import torch

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat"], images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)

#print(f"text_embeds shape: {outputs['text_embeds'].shape}")
#print(f"image_embeds shape: {outputs['image_embeds'].shape}")

model_input = torch.cat([outputs['text_embeds'], outputs['image_embeds']], dim=1)
model_to_test = MyModel(2, 4, 10)

miao, miaomiao = model_to_test(model_input)

print(f"head_task_a: {miao}")
print(f"head_task_b: {miaomiao}")


head_task_a: tensor([[-0.0186, -0.0432]], grad_fn=<AddmmBackward0>)
head_task_b: tensor([[ 0.0063, -0.0361,  0.0323,  0.0150]], grad_fn=<AddmmBackward0>)


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/MAMI DATASET-20240614T064502Z-001.zip"

Archive:  /content/drive/MyDrive/MAMI DATASET-20240614T064502Z-001.zip
  inflating: MAMI DATASET/training.zip  
  inflating: MAMI DATASET/readme.txt  
  inflating: MAMI DATASET/test_labels.txt  
  inflating: MAMI DATASET/trial.zip  
  inflating: MAMI DATASET/test.zip   


In [3]:
!unzip "/content/MAMI DATASET/training.zip"

Output streaming troncato alle ultime 5000 righe.
  inflating: TRAINING/4665.jpg       
  inflating: TRAINING/4666.jpg       
  inflating: TRAINING/4667.jpg       
  inflating: TRAINING/4669.jpg       
  inflating: TRAINING/467.jpg        
  inflating: TRAINING/4670.jpg       
  inflating: TRAINING/4671.jpg       
  inflating: TRAINING/4673.jpg       
  inflating: TRAINING/4674.jpg       
  inflating: TRAINING/4675.jpg       
  inflating: TRAINING/4678.jpg       
  inflating: TRAINING/4679.jpg       
  inflating: TRAINING/468.jpg        
  inflating: TRAINING/4680.jpg       
  inflating: TRAINING/4681.jpg       
  inflating: TRAINING/4682.jpg       
  inflating: TRAINING/4683.jpg       
  inflating: TRAINING/4685.jpg       
  inflating: TRAINING/4686.jpg       
  inflating: TRAINING/4687.jpg       
  inflating: TRAINING/4688.jpg       
  inflating: TRAINING/4689.jpg       
  inflating: TRAINING/469.jpg        
  inflating: TRAINING/4690.jpg       
  inflating: TRAINING/4691.jpg       


In [ ]:
#CLIP Image preprocessing and embedding

import os
import json
from tqdm import tqdm
from PIL import Image
training_images_path= "/content/TRAINING"

with open("/content/drive/MyDrive/train_image_text.json","r") as f:
  data = json.load(f)

  text_list=[]
  image_list=[]
  i=0
  for item in tqdm(data):
    if i>4500:
      break
    file_path = f"/content/TRAINING/{item['file_name']}"
    image_text = item["text"]
    text_list.append(image_text)
    image_list.append(Image.open(f"{file_path}"))
    i+=1
  inputs = processor(text=text_list, images=image_list, return_tensors="pt", padding=True)




 50%|█████     | 4501/9000 [00:00<00:00, 10823.66it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (343 > 77). Running this sequence through the model will result in indexing errors


In [17]:
classifier = MyModel(5)


In [1]:
inputs["input_ids"].shape


NameError: name 'inputs' is not defined